In [58]:
# params = {"wp": ['CHEAPEST', 'ECOFRIENDLY', 'SHORTEST']}
# wp_string = "(1,1,1,1)" # Set the default value of wp_string
# if "wp" in params: # Check if there is "wp" node in params
#     wp_list = params["wp"] # Get the value of "wp" node, which should be a list
#     for i, value in enumerate(wp_list): # Loop over the elements of the list
#         if value == "CHEAPEST": # Check if the element is CHEAPEST
#             wp_string = "(2," + wp_string[3:] # Substitute the second character of wp_string with "2"
#         elif value == "ECOFRIENDLY": # Check if the element is ECOFRIENDLY
#             wp_string = wp_string[:3] + "2" + wp_string[4:] # Substitute the fourth character of wp_string with "2"
#         elif value == "FASTEST": # Check if the element is FASTEST
#             wp_string = wp_string[:5] + "2" + wp_string[6:] # Substitute the sixth character of wp_string with "2"
#         elif value == "SHORTEST": # Check if the element is SHORTEST
#             wp_string = wp_string[:7] + "2)" # Substitute the eighth character of wp_string with "2"
#     params["wp"] = wp_string # Substitute the value of "wp" node in params with wp_string

# params

In [59]:
import json
import pandas as pd
import polyline
import numpy as np

def filter_highest_weighted_score(group):
    highest_score = group["weightedScore"].max()
    return group[group["weightedScore"] == highest_score]

def process_json_file(json_file):
    with open(json_file) as json_data:
        j_data = json.load(json_data)
    
    groups_df = pd.DataFrame(j_data['groups'])
    groups_df = groups_df.explode("trips")
    groups_df['group'] = groups_df.index
    groups_df = groups_df.reset_index()
    groups_df = groups_df[['group', 'trips']]
    
    trips_df = pd.json_normalize(groups_df['trips'])
    trips_df = trips_df[["id", "segments", "depart", "arrive", "weightedScore"]]
    trips_df = trips_df.rename(columns={"id": "trip_id"})
    
    groups_df = pd.concat([groups_df.drop('trips', axis=1), trips_df], axis=1)
    groups_df = groups_df.explode("segments").reset_index()
    
    filtered_df = groups_df.groupby("group").apply(filter_highest_weighted_score).reset_index(drop=True)
    groups_df = filtered_df.copy()

    normalized_segments = pd.json_normalize(groups_df['segments'])
    normalized_segments = normalized_segments.add_prefix('segments.')
    
    groups_df = pd.concat([groups_df.drop(['index', 'segments'], axis=1), normalized_segments], axis=1)
    
    return groups_df,j_data

def modify_params(self, params):
    wp_string = "(1,1,1,1)" # Set the default value of wp_string
    if "wp" in params: # Check if there is "wp" node in params
        wp_list = params["wp"] # Get the value of "wp" node, which should be a list
        for i, value in enumerate(wp_list): # Loop over the elements of the list
            if value == "CHEAPEST": # Check if the element is CHEAPEST
                wp_string = "(2," + wp_string[3:] # Substitute the second character of wp_string with "2"
            elif value == "ECOFRIENDLY": # Check if the element is ECOFRIENDLY
                wp_string = wp_string[:3] + "2" + wp_string[4:] # Substitute the fourth character of wp_string with "2"
            elif value == "FASTEST": # Check if the element is FASTEST
                wp_string = wp_string[:5] + "2" + wp_string[6:] # Substitute the sixth character of wp_string with "2"
            elif value == "SHORTEST": # Check if the element is SHORTEST
                wp_string = wp_string[:7] + "2)" # Substitute the eighth character of wp_string with "2"
        params["wp"] = wp_string # Substitute the value of "wp" node in params with wp_string
    return params # Return the modified params


def decode_polyline(segmentTemplates_df):
    try:
        for i, value in enumerate(segmentTemplates_df["streets"]):
            if value is not np.nan:
                for item in value:
                    encoded_waypoints = item['encodedWaypoints']
                    decoded_waypoints = polyline.decode(encoded_waypoints)
                    item['encodedWaypoints'] = decoded_waypoints
    except:
        pass

    try:
        for i, value in enumerate(segmentTemplates_df["shapes"]):
            if value is not np.nan:
                for item in value:
                    encoded_waypoints = item['encodedWaypoints']
                    decoded_waypoints = polyline.decode(encoded_waypoints)
                    item['encodedWaypoints'] = decoded_waypoints
    except:
        pass

    segmentTemplates_df = segmentTemplates_df.add_prefix('segments.')
    return segmentTemplates_df

def convert_timestamp_columns(df):
    try:
        # Find the timezone
        timezone = df['segments.from.timezone'][df['segments.from.timezone'].first_valid_index()]

        # Convert 'depart' column
        try:
            df['depart'] = pd.to_datetime(df['depart'], unit='s').dt.tz_localize('UTC').dt.tz_convert(timezone).dt.strftime('%Y-%m-%d %H:%M:%S')
        except Exception as e:
            print(f"An error occurred during 'depart' column conversion: {str(e)}")

        # Convert 'arrive' column
        try:
            df['arrive'] = pd.to_datetime(df['arrive'], unit='s').dt.tz_localize('UTC').dt.tz_convert(timezone).dt.strftime('%Y-%m-%d %H:%M:%S')
        except Exception as e:
            print(f"An error occurred during 'arrive' column conversion: {str(e)}")

        # Convert 'segments.endTime' column
        try:
            df['segments.endTime'] = pd.to_datetime(df['segments.endTime'], unit='s').dt.tz_localize('UTC').dt.tz_convert(timezone).dt.strftime('%Y-%m-%d %H:%M:%S')
        except Exception as e:
            print(f"An error occurred during 'segments.endTime' column conversion: {str(e)}")

        # Convert 'segments.startTime' column
        try:
            df['segments.startTime'] = pd.to_datetime(df['segments.startTime'], unit='s').dt.tz_localize('UTC').dt.tz_convert(timezone).dt.strftime('%Y-%m-%d %H:%M:%S')
        except Exception as e:
            print(f"An error occurred during 'segments.startTime' column conversion: {str(e)}")

    except Exception as e:
        print(f"An error occurred during timestamp conversion: {str(e)}")

    return df

def create_nested_dict( row, column_name, value):
        if '.' in column_name:
            main_node, subnodes = column_name.split('.', 1)
            if main_node not in row:
                row[main_node] = {}
            create_nested_dict(row[main_node], subnodes, value)
        else:
            row[column_name] = value

def to_json(df):
        json_list = []
        for _, row in df.iterrows():
            json_obj = {}
            for column_name, value in row.items():
                create_nested_dict(json_obj, column_name, value)
            json_list.append(json_obj)

        return json_list

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

def convert_dataframe_to_json(joined_df):
    # Grouping and converting to JSON
    grouped = joined_df.groupby(['group', 'trip_id', 'depart', 'arrive', 'weightedScore'])

    result = []
    for key, group in grouped:
        group_data = group.drop(columns=['group', 'trip_id', 'depart', 'arrive', 'weightedScore'])
        group_data = group_data.groupby(['segments.id'])
        segments=[]
        for i, seg in group_data:
            seg_data = seg.drop(columns=["segments.id"])
            j_d=to_json(seg_data)[0]['segments']
            seg_json = {
                "segment_id": i[0],
                "segment_data": j_d
            }
            segments.append(json.dumps(seg_json))

        group_dict = {
            "group": key[0],
            "trip_id": key[1],
            "depart": key[2],
            "arrive": key[3],
            "weightedScore": key[4],
            "segments": segments
        }
        result.append(group_dict)
    for i,item in enumerate(result):
        result[i] = (json.dumps(item, cls=NpEncoder)).replace('\\"', '"')
    return result


def create_new_frame(joined_df, arrive_col, depart_col, prefix=''):
    # Convert the 'arrive' and 'depart' columns to datetime data type
    joined_df[arrive_col] = pd.to_datetime(joined_df[arrive_col])
    joined_df[depart_col] = pd.to_datetime(joined_df[depart_col])

    # Create a new DataFrame by concatenating all columns
    newframe = pd.concat([
        joined_df,
        (joined_df[arrive_col] - joined_df[depart_col]).dt.total_seconds() / 60
    ], axis=1)

    # Assign appropriate column names
    new_column_name = prefix + 'duration'
    newframe.columns = [*joined_df.columns, new_column_name]

    # Convert column types
    newframe[new_column_name] = newframe[new_column_name].astype(int)
    newframe[arrive_col] = newframe[arrive_col].astype(str)
    newframe[depart_col] = newframe[depart_col].astype(str)

    return newframe

In [60]:
# api_url ="https://api.tripgo.com/v1/routing.json?v={}&from={}&to={}&modes={}".format(params['v'],params['from'],params['to'],params['modes'])
# print(api_url)
# response = requests.get(api_url, headers={"X-TripGo-Key": api_key}) 

groups_df, j_data = process_json_file('new_example.json')

In [61]:
# # Order the 'column_name' column in descending order
# groups_df.sort_values('weightedScore', ascending=False).nlargest(9, 'weightedScore')

In [62]:
segmentTemplates_df = pd.json_normalize(j_data['segmentTemplates'])
segmentTemplates_df = decode_polyline(segmentTemplates_df)

In [63]:
joined_df = groups_df.merge(segmentTemplates_df,
                            left_on='segments.segmentTemplateHashCode',
                            right_on='segments.hashCode',
                            how='left')

In [64]:
# # Convert all float columns to string type
# float_columns = joined_df.select_dtypes(include=[np.float64]).columns
# joined_df[float_columns] = joined_df[float_columns].astype(str)

In [65]:
joined_df = convert_timestamp_columns(joined_df)
# joined_df = joined_df.rename(columns={"segments.id": "segments_id"})

In [66]:
# joined_df[(joined_df["trip_id"] == '70858b20-0d82-4d50-9239-cbc03b74c96f') | (joined_df["trip_id"] == '91cc3701-109a-4f5d-82f0-03feaffe35f8')]

In [67]:
joined_df = create_new_frame(joined_df, arrive_col='arrive', depart_col='depart', prefix='')
joined_df = create_new_frame(joined_df, arrive_col='segments.endTime', depart_col='segments.startTime', prefix='segments.')

In [68]:
# joined_df[(joined_df["group"] == 0) & (joined_df['segments.modeIdentifier'] == 'pt_pub')][['group','trip_id','depart','arrive','weightedScore','segments.serviceNumber','segments.modeInfo.alt','segments.shapes','segments.from.address','segments.to.address']]

In [69]:
# joined_df[(joined_df["group"] == 0) & (joined_df["trip_id"] == "753b8387-2873-43f9-b0ae-14ff70a86aed")]

In [70]:
# grouped = joined_df.groupby(['group', 'trip_id'])

# # Sort the groups by "segments.startTime" in ascending order
# grouped = grouped.apply(lambda x: x.sort_values("segments.startTime", ascending=True))
# # Reset the index of the sorted DataFrame
# grouped = grouped.reset_index(drop=True)
# grouped = grouped.copy()
# grouped[(grouped["group"] == 0) & (grouped["trip_id"] == "753b8387-2873-43f9-b0ae-14ff70a86aed")]

In [57]:
joined_df.columns

Index(['group', 'trip_id', 'depart', 'arrive', 'weightedScore',
       'segments.availability', 'segments.endTime', 'segments.id',
       'segments.segmentTemplateHashCode', 'segments.startTime',
       ...
       'segments.location.popularity', 'segments.location.publicTransportMode',
       'segments.location.region', 'segments.location.services',
       'segments.location.stopCode', 'segments.location.stopType',
       'segments.location.timezone', 'segments.location.wheelchairAccessible',
       'duration', 'segments.duration'],
      dtype='object', length=118)

In [45]:
joined_df[["group","trip_id","segments.id","segments.startTime"]]

,group,trip_id,segments.id,segments.startTime
0,0,dbeb68a9-dcd5-48a2-b3d2-d514ae4db22a,G6xHTiRv,2023-06-05 11:12:43
1,0,dbeb68a9-dcd5-48a2-b3d2-d514ae4db22a,Rm2HH8Bf,2023-06-05 11:26:00
2,0,dbeb68a9-dcd5-48a2-b3d2-d514ae4db22a,Tx5WUfe5,2023-06-05 11:36:00
3,1,97c3cfe7-2ca3-471c-9d75-f9a3fd169dcf,WkyjfuTS,2023-06-05 11:27:22
4,1,97c3cfe7-2ca3-471c-9d75-f9a3fd169dcf,4Lq5uR2i,2023-06-05 11:35:23
5,1,97c3cfe7-2ca3-471c-9d75-f9a3fd169dcf,MfbSNICH,2023-06-05 11:40:30
6,1,97c3cfe7-2ca3-471c-9d75-f9a3fd169dcf,8ofTUrb1,2023-06-05 11:44:00
7,1,97c3cfe7-2ca3-471c-9d75-f9a3fd169dcf,9j0D1mAt,2023-06-05 11:48:00
8,2,278d9aa5-e508-4e0f-ae32-5d71d7d00d82,aKBzb8GC,2023-06-05 11:25:11
9,2,278d9aa5-e508-4e0f-ae32-5d71d7d00d82,bDbeXIYI,2023-06-05 11:45:59


In [53]:
grouped = joined_df.groupby(['group', 'trip_id', 'depart', 'arrive','duration', 'weightedScore'])

result = []
for key, group in grouped:
    group_data = group.drop(columns=['group', 'trip_id', 'depart', 'arrive','duration', 'weightedScore'])
    group_data = group_data.groupby(['segments.id'])
    # group_data.obj = group_data.obj.sort_values('segments.startTime', ascending=True)
    # group_data.obj = group_data.obj.reset_index(drop=True)
    segments=[]
    for i, seg in group_data:
        seg_data = seg.drop(columns=["segments.id"])
        j_d=to_json(seg_data)[0]['segments']
        seg_json = {
            "segment_id": i[0],
            "segment_data": j_d
        }
        segments.append(seg_json)
    # Sort the list of dictionaries by "startTime"
    segments_sorted = sorted(segments, key=lambda x: x['segment_data']['startTime'])

    group_dict = {
        "group": key[0],
        "trip_id": key[1],
        "depart": key[2],
        "arrive": key[3],
        "duration": key[4],
        "weightedScore": key[5],
        "segments": segments_sorted
    }
    result.append(group_dict)

In [ ]:
"2023-06-05 11:25:11",
"2023-06-05 11:45:59",
"2023-06-05 11:53:00"

In [56]:
json.dumps(segments_sorted)

'[{"segment_id": "aKBzb8GC", "segment_data": {"availability": "AVAILABLE", "endTime": "2023-06-05 11:45:59", "segmentTemplateHashCode": -371659303, "startTime": "2023-06-05 11:25:11", "routeDirection": NaN, "routeID": NaN, "routeId": NaN, "serviceDirection": NaN, "serviceName": NaN, "serviceNumber": NaN, "serviceShortName": NaN, "serviceTripID": NaN, "stops": NaN, "ticketWebsiteURL": NaN, "wheelchairAccessible": NaN, "externalData": {"directionId": NaN, "operatorName": NaN, "routeId": NaN, "stopId": NaN, "type": NaN}, "serviceTextColor": {"blue": NaN, "green": NaN, "red": NaN}, "serviceColor": {"blue": NaN, "green": NaN, "red": NaN}, "action": "<TIME>: Walk", "hashCode": -371659303, "metres": 1418.0, "modeIdentifier": "wa_wal", "notes": "<DURATION> \\u22c5 1.5 km", "streets": [{"encodedWaypoints": [[45.47703, 9.17825], [45.4771, 9.17861], [45.47733, 9.18001], [45.47736, 9.18024], [45.47743, 9.18073], [45.47744, 9.18077], [45.47743, 9.1808], [45.47748, 9.18094], [45.47748, 9.18094]], "i

In [27]:
seg

,segments.availability,segments.endTime,segments.id,segments.segmentTemplateHashCode,segments.startTime,segments.routeID,segments.routeId,segments.serviceDirection,segments.serviceName,segments.serviceNumber,...,segments.from.modeInfo.identifier,segments.from.modeInfo.localIcon,segments.from.name,segments.from.popularity,segments.from.publicTransportMode,segments.from.services,segments.from.stopCode,segments.from.stopType,segments.from.wheelchairAccessible,segments.duration
9,AVAILABLE,2023-05-23 14:56:00,bDtp8Wea,2006469984,2023-05-23 14:33:40,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22


In [31]:
group_data.obj

,segments.availability,segments.endTime,segments.id,segments.segmentTemplateHashCode,segments.startTime,segments.routeID,segments.routeId,segments.serviceDirection,segments.serviceName,segments.serviceNumber,...,segments.from.modeInfo.identifier,segments.from.modeInfo.localIcon,segments.from.name,segments.from.popularity,segments.from.publicTransportMode,segments.from.services,segments.from.stopCode,segments.from.stopType,segments.from.wheelchairAccessible,segments.duration
0,AVAILABLE,2023-05-23 15:55:23,1xeFX185,-1453697125,2023-05-23 15:45:27,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,AVAILABLE,2023-05-23 16:00:30,ur17Dk1c,-362314877,2023-05-23 15:55:23,M2,M2,cascina gobba,m2 - linea verde,2,...,pt_pub_subway,subway,Moscova,1276.0,pt_pub_subway,,MOSCOVA,subway,False,5
2,AVAILABLE,2023-05-23 16:08:12,4oLqoTGr,-1954877621,2023-05-23 16:00:30,NaN,NaN,NaN,NaN,NaN,...,pt_pub_subway,subway,Centrale Fs,2424.0,pt_pub_subway,,CENTRALE FS,subway,True,7


In [18]:
grouped

,group,trip_id,depart,arrive,weightedScore,segments.availability,segments.endTime,segments.id,segments.segmentTemplateHashCode,segments.startTime,...,segments.from.modeInfo.localIcon,segments.from.name,segments.from.popularity,segments.from.publicTransportMode,segments.from.services,segments.from.stopCode,segments.from.stopType,segments.from.wheelchairAccessible,duration,segments.duration
0,0,91cc3701-109a-4f5d-82f0-03feaffe35f8,2023-05-23 15:45:27,2023-05-23 16:08:12,36.4,AVAILABLE,2023-05-23 15:55:23,1xeFX185,-1453697125,2023-05-23 15:45:27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22,9
1,0,91cc3701-109a-4f5d-82f0-03feaffe35f8,2023-05-23 15:45:27,2023-05-23 16:08:12,36.4,AVAILABLE,2023-05-23 16:00:30,ur17Dk1c,-362314877,2023-05-23 15:55:23,...,subway,Moscova,1276.0,pt_pub_subway,,MOSCOVA,subway,False,22,5
2,0,91cc3701-109a-4f5d-82f0-03feaffe35f8,2023-05-23 15:45:27,2023-05-23 16:08:12,36.4,AVAILABLE,2023-05-23 16:08:12,4oLqoTGr,-1954877621,2023-05-23 16:00:30,...,subway,Centrale Fs,2424.0,pt_pub_subway,,CENTRALE FS,subway,True,22,7
3,1,39cc40ea-fd74-49c3-a0fd-87ec0b798229,2023-05-23 15:33:28,2023-05-23 16:02:21,52.3,AVAILABLE,2023-05-23 15:36:23,qRcTw1PE,-407696571,2023-05-23 15:33:28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,2
4,1,39cc40ea-fd74-49c3-a0fd-87ec0b798229,2023-05-23 15:33:28,2023-05-23 16:02:21,52.3,AVAILABLE,2023-05-23 15:55:59,qkp0HV7c,1373327082,2023-05-23 15:36:23,...,bus,v.le elvezia (arena),1611.0,pt_pub_bus,,11307,bus,False,28,19
5,1,39cc40ea-fd74-49c3-a0fd-87ec0b798229,2023-05-23 15:33:28,2023-05-23 16:02:21,52.3,AVAILABLE,2023-05-23 16:02:21,pBJk4OTJ,550665666,2023-05-23 15:55:59,...,bus,sondrio m3,1242.0,pt_pub_bus,,11911,bus,False,28,6
6,2,e36c62ad-2cf5-48c7-b323-34b94ef49ab0,2023-05-23 15:39:20,2023-05-23 16:08:04,46.5,AVAILABLE,2023-05-23 15:56:57,wzsWTLQK,-140434237,2023-05-23 15:39:20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,17
7,2,e36c62ad-2cf5-48c7-b323-34b94ef49ab0,2023-05-23 15:39:20,2023-05-23 16:08:04,46.5,AVAILABLE,2023-05-23 16:06:09,BnOKymyM,2047788925,2023-05-23 15:56:57,...,tram,v.le m.te grappa via gioia,1214.0,pt_pub_tram,,17331,tram,NaN,28,9
8,2,e36c62ad-2cf5-48c7-b323-34b94ef49ab0,2023-05-23 15:39:20,2023-05-23 16:08:04,46.5,AVAILABLE,2023-05-23 16:08:04,t1fheyc0,1704507292,2023-05-23 16:06:09,...,tram,stazione centrale via tonale,2558.0,pt_pub_tram,,11930,tram,True,28,1
9,3,cc582871-b7fe-40b8-8711-e5fed8435fd3,2023-05-23 14:33:40,2023-05-23 15:12:04,136.9,AVAILABLE,2023-05-23 14:56:00,bDtp8Wea,2006469984,2023-05-23 14:33:40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38,22


In [177]:
vars(group_data)

{'_selection': None,
 'level': None,
 'as_index': True,
 'keys': ['segments.id'],
 'sort': True,
 'group_keys': True,
 'observed': False,
 'dropna': True,
 'obj':     segments.availability     segments.endTime segments.id   
 174             AVAILABLE  2023-05-23 15:56:00    dDfy9Zfg  \
 175             AVAILABLE  2023-05-23 16:07:00    nLVtgd2e   
 176             AVAILABLE  2023-05-23 16:12:04    6a70yUUr   
 
      segments.segmentTemplateHashCode   segments.startTime segments.routeID   
 174                        2006469984  2023-05-23 15:33:40              NaN  \
 175                       -1680111054  2023-05-23 15:56:00              XP2   
 176                       -1303860011  2023-05-23 16:07:00              NaN   
 
     segments.routeId segments.serviceDirection      segments.serviceName   
 174              NaN                       NaN                       NaN  \
 175              XP2           Milano Centrale  Malpensa-Milano Centrale   
 176              NaN          

In [84]:
column_lst=[]
for i in joined_df.columns:
    column_lst.append(i)
print(column_lst)

['group', 'trip_id', 'depart', 'arrive', 'weightedScore', 'segments.availability', 'segments.endTime', 'segments.id', 'segments.segmentTemplateHashCode', 'segments.startTime', 'segments.routeDirection', 'segments.routeId', 'segments.serviceDirection', 'segments.serviceName', 'segments.serviceNumber', 'segments.serviceShortName', 'segments.serviceTripID', 'segments.stops', 'segments.ticketWebsiteURL', 'segments.wheelchairAccessible', 'segments.externalData.directionId', 'segments.externalData.operatorName', 'segments.externalData.routeId', 'segments.externalData.stopId', 'segments.externalData.type', 'segments.serviceTextColor.blue', 'segments.serviceTextColor.green', 'segments.serviceTextColor.red', 'segments.serviceColor.blue', 'segments.serviceColor.green', 'segments.serviceColor.red', 'segments.action', 'segments.hashCode', 'segments.metres', 'segments.modeIdentifier', 'segments.notes', 'segments.streets', 'segments.travelDirection', 'segments.type', 'segments.visibility', 'segments

In [94]:
json_data = convert_dataframe_to_json(joined_df)

TypeError: Object of type int64 is not JSON serializable

In [69]:
print(json_data)

[{"group": 0, "trip_id": "1b0f1eb9-7248-40d1-84d3-fceb9c9b06f3", "depart": "2023-05-22 18:10:47", "arrive": "2023-05-22 18:48:08", "weightedScore": "20.7", "segments": {"availability": "AVAILABLE", "endTime": "2023-05-22 18:12:58", "id": "btw0Edar", "segmentTemplateHashCode": -316501168, "startTime": "2023-05-22 18:10:47", "routeDirection": "nan", "routeId": "B78", "serviceDirection": "via govone", "serviceName": "bisceglie m1 - govone", "serviceNumber": "78", "serviceShortName": "", "serviceTripID": "20230508_1_LUNEDI_Intermedio_80_deviata_f5i_10492_35924575", "stops": "nan", "ticketWebsiteURL": "http://www.atm.it/", "wheelchairAccessible": false, "externalData": {"directionId": "nan", "operatorName": "ATM", "routeId": "B78", "stopId": "10275", "type": "gtfs"}, "serviceTextColor": {"blue": "nan", "green": "nan", "red": "nan"}, "serviceColor": {"blue": "nan", "green": "nan", "red": "nan"}, "action": "<TIME>: Walk", "hashCode": -316501168, "metres": "151.0", "modeIdentifier": "wa_wal", 

In [32]:
def _build_api_url(params):
        base_url = "https://api.tripgo.com/v1/routing.json?v=11"
        url_params = []

        for key, value in params.items():
            if key == "modes":
                for item in value:
                    url_params.append(f"modes={item}")
            else:
                url_params.append(f"{key}={value}")

        url = base_url + "&" + "&".join(url_params)
        return url



In [33]:
params = {
  "from": "(45.41425,9.40353)",
  "to": "(45.48485,9.20339)",
  "modes": ["pt_pub","pt_tax"]
}
_build_api_url(params)

'https://api.tripgo.com/v1/routing.json?v=11&from=(45.41425,9.40353)&to=(45.48485,9.20339)&modes=pt_pub&modes=pt_tax'

In [30]:
import requests
import json
api_key = "9806d2b6fe13bb29b55739cd9bf59436"

def get_trip(api_key, trip_id):
    url = f"https://api.tripgo.com/v1/trip/save/{trip_id}"
    headers = {'Accept': 'application/json',
           'X-TripGo-Key': api_key}
    # headers["Content-Type"] = "application/json"

    response = requests.get(url, headers=headers)
    return response
    # if response.status_code == 200:
    #     try:
    #         return response.json()
    #     except json.JSONDecodeError:
    #         print("Error: Invalid JSON response.")
    #         return None
    # else:
    #     print(f"Error: Request failed with status code {response.status_code}.")
    #     return None


In [31]:
id = "2b1f0d33-2931-4089-b89c-4f1914a08c85"
x = get_trip(api_key,id)

In [33]:
import requests

# Create a session
session = requests.Session()

# Set the base URL for TripGo API
base_url = 'https://api.tripgo.com/v1/'

# Set your TripGo API key
api_key = '9806d2b6fe13bb29b55739cd9bf59436'

# Set the trip ID to save
trip_id = '2b1f0d33-2931-4089-b89c-4f1914a08c85'

# Set the URL for saving the trip
save_trip_url = f'{base_url}trip/save/{trip_id}'

# Set the headers
headers = {
    'Accept': 'application/json',
    'X-TripGo-Key': api_key
}

# Send a POST request to save the trip
response = session.post(save_trip_url, headers=headers)

# Check the response
if response.status_code == 200:
    trip_data = response.json()
    trip_token = trip_data['token']
    trip_url = trip_data['url']
    trip_update_url = trip_data.get('updateUrl')

    print('Trip saved successfully!')
    print('Trip token:', trip_token)
    print('Trip URL:', trip_url)
    if trip_update_url:
        print('Trip update URL:', trip_update_url)
else:
    print('Failed to save the trip:', response.text)


Failed to save the trip: 


In [57]:
import requests
import json

# Create a session
session = requests.Session()

# Set the base URL for TripGo API
base_url = 'https://api.tripgo.com/v1/'

# Set your TripGo API key
api_key = '9806d2b6fe13bb29b55739cd9bf59436'

# Set the origin and destination coordinates
origin = 'LATITUDE,LONGITUDE'  # Replace with actual origin coordinates
destination = 'LATITUDE,LONGITUDE'  # Replace with actual destination coordinates

# Set the URL for retrieving trips
trips_url = f'{base_url}routing.json'

# Set the headers
headers = {
    'X-TripGo-Key': api_key
}

# Set the query parameters
params = {
    "v": 11,
    "from" : "(45.070850, 7.652193)",
    "to" : "(45.070092, 7.680763)",
    "modes" : "pt_pub"
}

# Send a GET request to retrieve the trips
response = session.get(trips_url, headers=headers, params=params)

# Check the response
if response.status_code == 200:
    trips_data = response.json()
    # Process the trips data as per your requirement
    print('Trips retrieved successfully!')
    # print(json.loads(trips_data))
else:
    print('Failed to retrieve trips:', response.text)

print(json.dumps(trips_data))

Trips retrieved successfully!
{"groups": [{"frequency": 10, "sources": [{"provider": {"name": "GTT Servizio Urbano"}}, {"disclaimer": "\u00a9 OpenStreetMap contributors\n\nMap data available under the Open Database License. For more information, see http://www.openstreetmap.org/copyright.", "provider": {"name": "OpenStreetMap", "website": "https://www.openstreetmap.org"}}], "trips": [{"arrive": 1685376251, "availability": "AVAILABLE", "caloriesCost": 19, "carbonCost": 0.3, "depart": 1685374819, "hassleCost": 2, "hideExactTimes": false, "id": "3876c91a-9a31-4e60-9dfc-6a522fd353eb", "mainSegmentHashCode": -809985494, "queryIsLeaveAfter": true, "queryMinimumTransferSeconds": 180, "queryTime": 1685370365, "segments": [{"availability": "AVAILABLE", "endTime": 1685375130, "id": "4qdRVN3E", "segmentTemplateHashCode": -1881870102, "startTime": 1685374819}, {"availability": "AVAILABLE", "endTime": 1685376117, "externalData": {"directionId": 1, "operatorName": "GTT Servizio Urbano", "routeId": "

In [58]:
import requests

# Assuming you already have an active session called "session"

# Set the base URL for TripGo API
base_url = 'https://api.tripgo.com/v1/'

# Set your TripGo API key
api_key = '9806d2b6fe13bb29b55739cd9bf59436'

# Set the trip ID to save
trip_id = '3876c91a-9a31-4e60-9dfc-6a522fd353eb'

# Set the URL for saving the trip
save_trip_url = f'{base_url}trip/save/{trip_id}'

# Set the headers
headers = {
    'Accept': 'application/json',
    'X-TripGo-Key': api_key
}

# Send a POST request to save the trip
response = session.post(save_trip_url, headers=headers)

# Check the response
if response.status_code == 200:
    trip_data = response.json()
    trip_token = trip_data['token']
    trip_url = trip_data['url']
    trip_update_url = trip_data.get('updateUrl')

    print('Trip saved successfully!')
    print('Trip token:', trip_token)
    print('Trip URL:', trip_url)
    if trip_update_url:
        print('Trip update URL:', trip_update_url)
else:
    print('Failed to save the trip:', response.text)


Failed to save the trip: 


In [59]:
vars(response)

{'_content': b'',
 '_content_consumed': True,
 '_next': None,
 'status_code': 405,
 'headers': {'Server': 'nginx', 'Date': 'Mon, 29 May 2023 14:27:12 GMT', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Allow': 'DELETE,POST,GET,OPTIONS,HEAD', 'X-TripGo-Server': 'recombination-local.skedgo.com', 'X-TripGo-Proxy-Path': '[recombination-local.skedgo.com, stars-api-gateway]'},
 'raw': <urllib3.response.HTTPResponse at 0x7f43382e2770>,
 'url': 'https://api.tripgo.com/v1/trip/save/3876c91a-9a31-4e60-9dfc-6a522fd353eb',
 'encoding': None,
 'history': [],
 'reason': '',
 'cookies': <RequestsCookieJar[]>,
 'elapsed': datetime.timedelta(microseconds=148849),
 'request': <PreparedRequest [POST]>,
 'connection': <requests.adapters.HTTPAdapter at 0x7f43382d4280>}

In [26]:
# df_bestOnly_False, _ = process_json_file("bestOnly_False.json")
# grouped_df_bestOnly_False = df_bestOnly_False.groupby("group")["trip_id"].nunique().reset_index()
# grouped_df_bestOnly_False = grouped_df_bestOnly_False.rename(columns={"trip_id": "unique_trip_count"})
# grouped_df_bestOnly_False

,group,unique_trip_count
0,0,20
1,1,25


In [46]:
df_bestOnly_True,_ = process_json_file("bestOnly_True.json")

In [47]:
df_bestOnly_True

,group,trip_id,depart,arrive,weightedScore,segments.availability,segments.endTime,segments.id,segments.segmentTemplateHashCode,segments.startTime,...,segments.ticketWebsiteURL,segments.wheelchairAccessible,segments.externalData.directionId,segments.externalData.operatorName,segments.externalData.routeId,segments.externalData.stopId,segments.externalData.type,segments.serviceTextColor.blue,segments.serviceTextColor.green,segments.serviceTextColor.red
0,0,268a253c-3f25-4182-b95f-84cd83b43a79,1685517133,1685519626,96.1,AVAILABLE,1685517264,FlfHwDRR,-316501168,1685517133,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,268a253c-3f25-4182-b95f-84cd83b43a79,1685517133,1685519626,96.1,AVAILABLE,1685518260,xbBGU0X5,-639911449,1685517264,...,http://www.atm.it/,False,1.0,ATM,B78,10275,gtfs,0.0,0.0,0.0
2,0,268a253c-3f25-4182-b95f-84cd83b43a79,1685517133,1685519626,96.1,AVAILABLE,1685518323,aNuzNlcN,-1429348800,1685518260,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,268a253c-3f25-4182-b95f-84cd83b43a79,1685517133,1685519626,96.1,AVAILABLE,1685518729,vlffApRd,1649621358,1685518323,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,268a253c-3f25-4182-b95f-84cd83b43a79,1685517133,1685519626,96.1,AVAILABLE,1685519461,mWnsQ4o9,-1650703676,1685518729,...,NaN,False,0.0,ATM,B57,10993,gtfs,0.0,0.0,0.0
5,0,268a253c-3f25-4182-b95f-84cd83b43a79,1685517133,1685519626,96.1,AVAILABLE,1685519626,O8VCSoSd,-853745903,1685519461,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,e43f58b9-8cca-4cde-9e78-34b2b527cdbe,1685518446,1685520343,43.1,AVAILABLE,1685518926,Qn9MzAcb,-1020706129,1685518446,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1,e43f58b9-8cca-4cde-9e78-34b2b527cdbe,1685518446,1685520343,43.1,AVAILABLE,1685520343,3ea9fLJ8,764006679,1685518926,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
grouped_df_bestOnly_True = df_bestOnly_True.groupby("group")[["trip_id"]].nunique().reset_index()
grouped_df_bestOnly_True = grouped_df_bestOnly_True.rename(columns={"trip_id": "unique_trip_count"})
grouped_df_bestOnly_True

,group,unique_trip_count
0,0,1
1,1,1


In [35]:
df_bestOnly_True["weightedScore"].max()

96.1

In [34]:
df_bestOnly_True[df_bestOnly_True["trip_id"]=="268a253c-3f25-4182-b95f-84cd83b43a79"]

,group,trip_id,depart,arrive,weightedScore,segments.availability,segments.endTime,segments.id,segments.segmentTemplateHashCode,segments.startTime,...,segments.ticketWebsiteURL,segments.wheelchairAccessible,segments.externalData.directionId,segments.externalData.operatorName,segments.externalData.routeId,segments.externalData.stopId,segments.externalData.type,segments.serviceTextColor.blue,segments.serviceTextColor.green,segments.serviceTextColor.red
0,0,268a253c-3f25-4182-b95f-84cd83b43a79,1685517133,1685519626,96.1,AVAILABLE,1685517264,FlfHwDRR,-316501168,1685517133,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,268a253c-3f25-4182-b95f-84cd83b43a79,1685517133,1685519626,96.1,AVAILABLE,1685518323,aNuzNlcN,-1429348800,1685518260,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,268a253c-3f25-4182-b95f-84cd83b43a79,1685517133,1685519626,96.1,AVAILABLE,1685518729,vlffApRd,1649621358,1685518323,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,268a253c-3f25-4182-b95f-84cd83b43a79,1685517133,1685519626,96.1,AVAILABLE,1685519461,mWnsQ4o9,-1650703676,1685518729,...,NaN,False,0.0,ATM,B57,10993,gtfs,0.0,0.0,0.0
5,0,268a253c-3f25-4182-b95f-84cd83b43a79,1685517133,1685519626,96.1,AVAILABLE,1685519626,O8VCSoSd,-853745903,1685519461,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,268a253c-3f25-4182-b95f-84cd83b43a79,1685517133,1685519626,96.1,AVAILABLE,1685518260,xbBGU0X5,-639911449,1685517264,...,http://www.atm.it/,False,1.0,ATM,B78,10275,gtfs,0.0,0.0,0.0
